In [1]:

def essai(x):
    return x+1


res=essai(1)

print(res)

def bis(y):
    return y+2
res=bis(1)
print(res) 

2
3


In [2]:
import time
import math
import tensorflow as tf
import numpy as np
from tensorflow.python.training.moving_averages import assign_moving_average
from scipy.stats import multivariate_normal as normal
from tensorflow.python.ops import control_flow_ops
from tensorflow import random_normal_initializer as norm_init
from tensorflow import random_uniform_initializer as unif_init
from tensorflow import constant_initializer as const_init

class SolveAllenCahn(object):
    def __init__(self, sess):
     self.sess = sess
     self.d = 100
     self.T = 0.3
     self.n_time = 20
     self.n_layer =4
     self.n_neuron = [self.d, self.d+10, self.d+10, self.d]
     self.batch_size = 64
     self.valid_size = 256
     self.n_maxstep = 4000
     self.n_displaystep = 100
     self.learning_rate = 5e-4
     self.Yini = [0.3, 0.6]
     self.h = (self.T+0.0)/self.n_time
     self.sqrth = math.sqrt(self.h)
     self.t_stamp = np.arange(0, self.n_time)*self.h
     self._extra_train_ops = []
     
    def train(self):
     start_time = time.time()
     self.global_step = tf.get_variable('global_step', [],initializer=tf.constant_initializer(1),trainable=False , dtype=tf.int32)
     trainable_vars = tf.trainable_variables()
     grads = tf.gradients(self.loss, trainable_vars)
     optimizer = tf.train.AdamOptimizer(self.learning_rate)
     apply_op = optimizer.apply_gradients(zip(grads, trainable_vars),global_step=self.global_step)
     train_ops = [apply_op] + self._extra_train_ops
     self.train_op = tf.group(*train_ops)
     self.loss_history =[]
     self.init_history =[]
     dW_valid , X_valid= self.sample_path(self.valid_size) 
     feed_dict_valid ={self.dW: dW_valid , self.X: X_valid , self.is_training: False}
     step=1
     self.sess.run(tf.global_variables_initializer())
     temp_loss = self.sess.run(self.loss, feed_dict=feed_dict_valid)
     temp_init = self.Y0.eval()[0]
     self.loss_history.append(temp_loss)
     self.init_history.append(temp_init)
     print("step: %5u, loss: %.4e, "%(0, temp_loss)+"Y0: %.4e,runtime: %4u"%(temp_init,time.time()-start_time+self.t_bd))
     for i in range(self.n_maxstep+1):
         step = self.sess.run(self.global_step)
         dW_train , X_train = self.sample_path(self.batch_size)
         self.sess.run(self.train_op,feed_dict={self.dW: dW_train, self.X: X_train,self.is_training: True})
         if  step % self.n_displaystep==0:
             temp_loss = self.sess.run(self.loss,feed_dict=feed_dict_valid)
             temp_init = self.Y0.eval()[0]
             self.loss_history.append(temp_loss)
             self.init_history.append(temp_init)
             print("step:%5u,loss:%.4e, " %(step,temp_loss), "Y0:% .4e,runtime:% 4u" %(temp_init,time.time()-start_time+self.t_bd))
         step += 1
     end_time = time.time()
     print("running time:%.3f s"%(end_time -start_time+self.t_bd))
             
    def build(self):
                 start_time = time.time()
                 self.dW =tf.placeholder(tf.float64 ,[None, self.d, self.n_time], name='dW')
                 self.X = tf.placeholder(tf.float64 ,[None, self.d, self.n_time+1],name='X')
                 self.is_training = tf.placeholder(tf.bool)
                 self.Y0 = tf.Variable(tf.random_uniform([1],minval=self.Yini[0],maxval=self.Yini[1],dtype=tf.float64));
                 self.Z0 = tf.Variable(tf.random_uniform([1, self.d],minval=-.1,maxval=.1,dtype=tf.float64))
                 self.allones = tf.ones(shape=[tf.shape(self.dW)[0], 1],dtype=tf.float64)
                 Y= self.allones * self.Y0
                 Z= tf.matmul(self.allones , self.Z0)
                 with tf.variable_scope('forward'):
                     for t in range(0,self.n_time-1):
                         Y= Y - self.f_tf(self.t_stamp[t],self.X[:, :, t], Y, Z)*self.h
                         Y= Y + tf.reduce_sum(Z*self.dW[:, :, t], 1,keep_dims=True)
                         Z = self._one_time_net(self.X[:, :, t+1],str(t+1))/self.d
                     Y = Y - self.f_tf(self.t_stamp[self.n_time -1],self.X[:, :, self.n_time -1],Y, Z)*self.h
                     Y = Y + tf.reduce_sum(Z*self.dW[:, :, self.n_time-1], 1,keep_dims=True)
                     term_delta = Y - self.g_tf(self.T,self.X[:, :, self.n_time])
                     self.clipped_delta = tf.clip_by_value(term_delta , -50.0, 50.0)
                     self.loss = tf.reduce_mean(self.clipped_delta**2)
                     self.t_bd = time.time()-start_time
                
    def sample_path(self, n_sample):
        dW_sample = np.zeros([n_sample , self.d, self.n_time])
        X_sample = np.zeros([n_sample, self.d, self.n_time+1])
        for i in range(self.n_time):
            dW_sample[:, :, i] = np.reshape(normal.rvs(mean=np.zeros(self.d),cov=1,size=n_sample)*self.sqrth,(n_sample, self.d))
            X_sample[:, :, i+1] = X_sample[:, :, i] + np.sqrt(2) * dW_sample[:, :, i]
        return dW_sample , X_sample
        
    def f_tf(self , t, X,  Y, Z):
        return Y-tf.pow(Y, 3)
    def g_tf(self, t, X):
       return 0.5/(1 + 0.2*tf.reduce_sum(X**2, 1, keep_dims=True))
   
    
    def _one_time_net(self, x, name):
       with tf.variable_scope(name):
           x_norm = self._batch_norm(x, name='layer0_normal')
           layer1 = self._one_layer(x_norm, self.n_neuron[1],name='layer1')
           layer2 = self._one_layer(layer1, self.n_neuron[2],name='layer2')
           z = self._one_layer(layer2, self.n_neuron[3], activation_fn=None, name='final')
       return z
       
        
    def _one_layer(self, input_, out_sz,activation_fn=tf.nn.relu,std=5.0, name='linear'):
            with tf.variable_scope(name):
                shape = input_.get_shape().as_list()
                w = tf.get_variable('Matrix',[shape[1], out_sz],tf.float64 , norm_init(stddev= std/np.sqrt(shape[1]+out_sz))) 
                hidden=tf.matmul(input_ , w)
                hidden_bn = self._batch_norm(hidden, name='normal')
            if activation_fn != None:
                    return activation_fn(hidden_bn)
            else:
                    return hidden_bn
                
    def _batch_norm(self, x, name):
            with tf.variable_scope(name):
                params_shape = [x.get_shape()[-1]]
                beta = tf.get_variable('beta', params_shape , tf.float64 ,norm_init(0.0, stddev=0.1,dtype=tf.float64))
                gamma =tf.get_variable('gamma', params_shape , tf.float64 ,unif_init(0.1, 0.5, dtype=tf.float64))
                mv_mean = tf.get_variable('moving_mean', params_shape,tf.float64,const_init(0.0, tf.float64), trainable=False)
                mv_var = tf.get_variable('moving_variance', params_shape,tf.float64,const_init(1.0, tf.float64), trainable=False)
                mean, variance = tf.nn.moments(x, [0], name='moments')
                self._extra_train_ops.append(assign_moving_average(mv_mean, mean, 0.99))
                self._extra_train_ops.append(assign_moving_average(mv_var , variance , 0.99))
                mean , variance = control_flow_ops.cond(self.is_training ,lambda: (mean, variance),lambda: (mv_mean , mv_var))
                y = tf.nn.batch_normalization(x, mean, variance, beta, gamma, 1e-6)
                y.set_shape(x.get_shape())
                return y
        
def main():
       tf.reset_default_graph()
       with tf.Session() as sess:
           tf.set_random_seed(1)
           print ("Begin to solve Allen -Cahn")
           model = SolveAllenCahn(sess)
           model.build()
           model.train()
           output = np.zeros((len(model.init_history), 3))
           output[:, 0] = np.arange(len(model.init_history))* model.n_displaystep
           output[:, 1] = model.loss_history
           output[:, 2] = model.init_history
           np.savetxt("./AllenCahn_d100.csv", output ,fmt=['%d', '%.5e', '%.5e'], delimiter=",",header="step, loss function, " +  "target value ,runtime", comments='')
if __name__ == '__main__':
               np.random.seed(1)
               main()



Begin to solve Allen -Cahn
Instructions for updating:
keep_dims is deprecated, use keepdims instead
step:     0, loss: 5.5935e-02, Y0: 3.5068e-01,runtime:   31
step:  100,loss:3.8714e-02,  Y0: 3.0230e-01,runtime:  58
step:  200,loss:2.6160e-02,  Y0: 2.5911e-01,runtime:  73
step:  300,loss:1.7261e-02,  Y0: 2.2066e-01,runtime:  88
step:  400,loss:1.1083e-02,  Y0: 1.8716e-01,runtime: 102
step:  500,loss:6.9629e-03,  Y0: 1.5833e-01,runtime: 116
step:  600,loss:4.1624e-03,  Y0: 1.3388e-01,runtime: 131
step:  700,loss:2.4662e-03,  Y0: 1.1369e-01,runtime: 146
step:  800,loss:1.4387e-03,  Y0: 9.7355e-02,runtime: 161
step:  900,loss:8.6324e-04,  Y0: 8.4553e-02,runtime: 175
step: 1000,loss:5.5017e-04,  Y0: 7.4734e-02,runtime: 189
step: 1100,loss:3.9184e-04,  Y0: 6.7450e-02,runtime: 204
step: 1200,loss:3.1778e-04,  Y0: 6.2431e-02,runtime: 218
step: 1300,loss:2.8410e-04,  Y0: 5.8817e-02,runtime: 232
step: 1400,loss:2.6393e-04,  Y0: 5.6359e-02,runtime: 247
step: 1500,loss:2.5306e-04,  Y0: 5.5006e-0